In [ ]:
from os import path,listdir
from datetime import datetime,timedelta
import time
import json
import requests
from stem import Signal
from stem.control import Controller
import re
import pandas as pd
import numpy as np
import random
import pytz
from tqdm import tqdm
local_tz = 'Asia/Almaty'

%load_ext autoreload
%autoreload 2

In [ ]:
def getone(pattern, string):
    m = pattern.search(string)
    res = m.group(1) if m else ''
    return res.strip()

def clean(txt):
    return txt.replace('&','').replace('\'','').strip()

def get_cookie_value(name):
    return [x['value'] for x in cookies if x['name']==name][0]

In [ ]:
base_url='https://sofascore.com/'
api_url='https://api.sofascore.com/api/v1/'

def generate_headers(referer):
    return {
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-US,en;q=0.5",
            "Connection": "keep-alive",
            "Host": "api.sofascore.com",
            "Origin": "https://www.sofascore.com",
            "Referer": referer,
            "TE": "Trailers",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0"
        }

def load_match_info(did, isft):
    load_json('lineups',did, headers)
    load_json('votes',did, headers, isft=isft)
    if isft==1:
        load_json('incidents',did, headers)
        load_json('graph',did, headers)
    #load_json('best-players',did, headers)
    #load_json('event',did, headers)
    #load_json('statistics',did, headers)
    #load_json('h2h',did, headers)
    #load_json('pregame-form',did, headers)
    #load_json('winning-odds',did, headers)

#

def load_json(fn, did, headers, isft=0):
    file_name='raw/{}_{}_{:%Y-%m-%d-%H%M}.json'.format(fn, did, datetime.now()) if fn=='votes' else f'raw/{fn}_{did}.json'
    if not path.exists(file_name) or (fn=='votes' and isft==0):
        script='' if fn=='event' else '/provider/1/'+fn if fn=='winning-odds' else '/'+fn
        link=f'{api_url}event/{did}{script}'
        r = requests.get(link, headers=headers)
        if r.status_code==200:
            with open(file_name, 'w+', encoding='utf8') as f:
                f.write(r.text)



In [ ]:
file_name='data/matches.csv'
df_matches=pd.read_csv(file_name, index_col=None)
#SERVER_TZ = 'UTC'
#df_matches['ts']=pd.DatetimeIndex(pd.to_datetime(df_matches['startTimestamp'], unit='s')).tz_localize(SERVER_TZ)
df_matches=df_matches.drop_duplicates(keep='first')

In [ ]:
df_matches_done=df_matches.loc[df_matches['done']==1]
df_matches=df_matches.loc[df_matches['done']<1]
df_matches_done.to_csv('data/matches_done.csv', index=False)
df_matches.to_csv('data/matches1.csv', index=False)


In [ ]:
exclude=['india', 'peru', 'south-africa', 'germany-amateur', 'saudi-arabia', 'united-arab-emirates']
df_matches=df_matches.loc[~df_matches['country'].isin(exclude)]

In [ ]:
df_matches.to_csv('data/matches1.csv', index=False)

In [ ]:
df_matches_new=pd.read_csv('data/matches1.csv', index_col=None).set_index('id')
df_matches=pd.read_csv('data/matches.csv', index_comatches


In [ ]:
df_matches.update(df_matches_new[['awayScoreHT','awayScoreET','homeScoreET','awayScoreFT','homeScoreFT','status','winnerCode','homeScoreHT']])
df_matches['id'] = df_matches.index
df_matches_new['id'] = df_matches_new.index
matchespd.concat([df_matches,df_matches_new]).drop_duplicates(subset='id', keep='first').reset_index(drop=True)

In [ ]:
LOCAL_TZ = 'Asia/Almaty'
df_matches['ts'].apply(lambda d: d.astimezone(pytz.timezone(LOCAL_TZ)))

In [ ]:
avoid_countries=['saudi-arabia', 'chile']
PROXY = "127.0.0.1:9051"

In [ ]:
from seleniumwire import webdriver
#from selenium.webdriver.chrome.options import Options

#webdriver.DesiredCapabilities.FIREFOX['proxy'] = {
#    "socksProxy": PROXY,
#    "socksVersion": 5,
#    "proxyType": "MANUAL",
#}

#options = webdriver.ChromeOptions()
#options.add_argument('--proxy-server=socks5://' + PROXY)
#chrome = webdriver.Chrome(executable_path=r'../lib/chromedriver.exe', options=options)
firefox = webdriver.Firefox(executable_path=r'../lib/geckodriver.exe')
firefox.scopes = [
    'fb.oddsportal.com/feed/match/*'
]
firefox.get("https://www.oddsportal.com/soccer/argentina/superliga/estudiantes-l-p-racing-club-C2xXOviA/")
#chrome.get("https://www.sofascore.com/")

#referer=firefox.current_url
#cookies=firefox.get_cookies()
#html = firefox.page_source


In [ ]:
request = firefox.requests[0]
parts=request.url.split('/')
id=parts[5].split('-')[2]
fn=f'raw/op/matches/{parts[4]}_{id}.json'
print(
    request.url,
    fn
)

In [ ]:
with open(fn, 'w+', encoding='utf8') as f:
    f.write(str(request.response.body))


In [ ]:
with open(file_name, 'w+', encoding='utf8') as f:
    f.write(firefox.page_source)

In [ ]:
def load_link(file_name, link):
    if path.exists(file_name):
        print(file_name, ' exists!')
        with open(file_name, 'r', encoding='utf8') as f:
            html=f.read()
    else:
        print(f'loading {link}...', end='')
        firefox.get('https://www.oddsportal.com/'+link)
        html = firefox.page_source
        time.sleep(random.uniform(0, 1))
        with open(file_name, 'w+', encoding='utf8') as f:
            f.write(html)
        print(f'done {len(html)} bytes')
    return html

def clear_html(file_name):
    html=''
    if path.exists(file_name):
        with open(file_name, 'r', encoding='utf8') as f:
            html=f.read()
        if '<!-- PAGE BODY -->' in html:
            html=html.split('<!-- PAGE BODY -->')[1]
            html=html.split('<!--  END PAGE BODY -->')[0]
            with open(file_name, 'w', encoding='utf8') as f:
                f.write(html)
    return html

In [ ]:
BASE_PATH='raw/op/days/'
pYears=r'<strong><a href="/soccer/([^/]+)/([^/]+)/results/">([^<]+)</a></strong>'
pNumbers=r'#/page/(\d+)/'
df_champs=pd.read_csv('data/op/champs.csv', index_col=None)
for row in df_champs[df_champs['active']==1].itertuples():
    link=row.link
    country=link.split('/')[2]
    champ=link.split('/')[3]
    file_name='{}{}_{}_{}.htm'.format(BASE_PATH,country,champ,1)
    html=load_link(file_name,link)
    
    mm=re.findall(pNumbers, html)
    if len(mm)>0:
        for x in mm:
            n=x[0]
            file_name='{}{}_{}_{}.htm'.format(BASE_PATH,country,champ,n)
            load_link(file_name,link+f'#/page/{n}/')
    mmy=re.findall(pYears, html)
    if len(mmy)>0:
        for y in mmy:
            country,champ,season=y
            if '2010' in season:
                break
            link=f'/soccer/{country}/{champ}/results/'
            file_name='{}{}_{}_{}.htm'.format(BASE_PATH,country,champ,1)
            html=load_link(file_name,link)
            mm=re.findall(pNumbers, html)
            if len(mm)>0:
                for x in mm:
                    n=x[0]
                    file_name='{}{}_{}_{}.htm'.format(BASE_PATH,country,champ,n)
                    load_link(file_name,link+f'#/page/{n}/')

    print(f' done {file_name}')
    #break

In [ ]:
DAYS_RAW_PATH='raw/op/days/'
for f in tqdm(listdir(DAYS_RAW_PATH)):
    print(f)
    fn=DAYS_RAW_PATH+f
    html=clear_html(fn)
    #break


In [ ]:
html

In [ ]:
d=datetime.strptime('2020-10-31', '%Y-%m-%d')
dstr='{:%Y-%m-%d}'.format(d)
matches['ts']=matches['match_time'].apply(lambda x: pytz.timezone(local_tz).localize(datetime.strptime(f'{dstr} {x}', '%Y-%m-%d %H:%M')))
matches.to_csv(f'raw/matches_{dstr}.csv')

In [ ]:
def load_matches(arr):
    l=len(arr)
    if l>0:
        did,isft=arr[0]
        print(f'Loading {did},{isft} from {l}...', end='')
        load_match_info(did, isft)
        print(' done!')
        time.sleep(random.uniform(1, 5))
        load_matches(np.delete(arr, 0, 0))

In [ ]:
load_match_info(1399303, 1)

In [ ]:
headers=generate_headers(referer)
load_matches(dids)

In [ ]:
datetime.utcfromtimestamp(1583791200)

# Days

In [ ]:
def tor_new_identity():
    with Controller.from_port(port=9052) as controller:
        controller.authenticate()
        #controller.authenticate(password="1234")
        controller.signal(Signal.NEWNYM)
        controller.close()

def load_days(test=False,c=0,n=0,max_load=random.randint(50, 100)):
    global dates,r,dstr,proxies
    l=len(dates)
    if l>0:
        dstr='{:%Y-%m-%d}'.format(dates[0])
        link=f'https://api.sofascore.com/api/v1/sport/football/scheduled-events/{dstr}'
        referer=f'https://www.sofascore.com/football/{dstr}'
        print(f'Loading {dstr} from {l}...', end='')
        headers=generate_headers(referer)
        r = requests.get(link, headers=headers, proxies=proxies)
        if test:
            print(r.status_code)
        if r.status_code==200:
            c=0
            with open(f'raw/{dstr}.json', 'w+', encoding='utf8') as f:
                f.write(r.text)
            n+=1
            if n>max_load:
                tor_new_identity()
                time.sleep(random.uniform(10, 20))
                load_days()
        else:
            print(f'ERROR {r.status_code}!!! Obtaining new identity. Try # {c}.')
            tor_new_identity()
            time.sleep(random.uniform(10, 20))
            c+=1
            if c>10:
                return 
            load_days(c=c)
        c=0
        print(f' done #{n} from the batch of {max_load}!')
        time.sleep(random.uniform(1, 5))
        dates=np.delete(dates, 0, 0)
        if not test:
            load_days(c=c,n=n,max_load=max_load)

In [ ]:
proxies = {'https': f'socks5://{PROXY}'}

dates=[]
d= datetime(2020, 10, 29)
end_date= datetime(2020, 11, 30)

while d<=end_date:
    dates.append(d)
    d+=timedelta(days=1)
dates=np.array(dates) 
np.random.shuffle(dates)

load_days(test=False)
#len(dates)

In [ ]:
from data_provider import DataProvider
ds='2014-01-01'
de='2020-10-31'

#ds='2020-04-05'
#de='2020-04-05'

dp=DataProvider()
#dp.load_days(ds, de)
#dp.load_days() # Process al files
dp.load_matches()

In [ ]:
#matches=[x for x in dp.DATA['events'] if 'coverage' in x.keys()]
dp.DATA

In [ ]:
[datetime.strptime(f.replace('.json', ''), '%Y-%m-%d') for f in listdir('raw/')] 

In [ ]:
file_name='data/matches1.csv'
dp.df_matches.to_csv(file_name, index=False)

In [ ]:
dp.COUNTER

In [ ]:
import json
file_name='raw/2016-08-28.json'
with open(file_name, 'r', encoding='utf8') as f:
    data=json.load(f)
matches=[x for x in data['events'] if 'coverage' in x.keys()]
matches=[x for x in matches if x['coverage']>-1]
matches=[x for x in matches if not x['status']['code'] in [60,70]]
len(matches)

In [ ]:
[x for x in matches if not x['status']['code']==100 and not x['status']['code']==110 and not x['status']['code']==0]

In [ ]:
matches=[x for x in dp.data['events'] if 'coverage' in x.keys()]
matches=[x for x in matches if x['coverage']>-1]
matches=[x for x in matches if x['status']['code']==100 or x['status']['code']==110]
len(matches)

In [ ]:
a=[  {'tournament' : x['tournament']['slug'],
    'country' : x['tournament']['category']['slug'],
    'round' : x['roundInfo']['round'] if 'roundInfo' in x.keys() else np.NaN,
    'status' : x['status']['code'],
    'homeTeam' : x['homeTeam']['name'],
    'homeTeamShort' : x['homeTeam']['shortName'],
    'awayTeam' : x['awayTeam']['name'],
    'awayTeamShort' : x['awayTeam']['shortName'],
    'homeScoreFT' : x['homeScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'homeScoreET' : x['homeScore']['current'] if 'current' in x.keys() else np.NaN,
    'homeScoreHT' : x['homeScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'awayScoreFT' : x['awayScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'awayScoreET' : x['awayScore']['current'] if 'current' in x.keys() else np.NaN,
    'awayScoreHT' : x['awayScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'id' : x['id'],
    'startTimestamp' : x['startTimestamp'],
    'coverage' : x['coverage'],
    'winnerCode' : x['winnerCode']} for x in matches]

In [ ]:
for i in range(10):
    print(i)

In [ ]:
load_json('best-players',did, headers)

# Fbref

In [ ]:
%load_ext autoreload
%autoreload 2

from fbref_parser import FbrefParser

fp=FbrefParser()
fp.parse_days()


In [ ]:
fp.DATA

In [ ]:
from data_provider import DataProvider

dp=DataProvider()
dp.load_fbref_matches()

## Match

In [78]:
def getone(pattern, string):
    m=re.search(pattern, string)
    return '' if not m else m.groups()[0].replace('&nbsp;&nbsp;',' ').replace('&nbsp;',' ').strip()

def inbetween(txt,st,end):
    print(st,end)
    return txt.split(st)[1].split(end)[0]

def gk(matches, tid):
    pid, pname, nationality,age,minutes,shots_on_target_against,goals_against_gk,saves= m.groups()
    return {
        'tid':tid2, 
        'pid':pid,
        'pname':pname,
        'nationality':nationality,
        'age':age,
        'minutes':minutes,
        'shots_on_target_against':shots_on_target_against,
        'goals_against_gk':goals_against_gk,
        'saves':saves
    }
def pl(matches, tid):
    players=[]
    for x in matches:
        players.append({
            'tid':tid, 
            'pid':x[0],
            'nameen':x[1],
            'name':x[2],
            'position':x[3],
            'nationality':x[4],
            'age':x[5],
            'goals':x[6],
            'assists':x[7],
            'pens_made':x[8],
            'pens_att':x[9],
            'shots_total':x[10],
            'shots_on_target':x[11],
            'cards_yellow':x[12],
            'cards_red':x[13],
            'fouls':x[14],
            'fouled':x[15],
            'offsides':x[16],
            'crosses':x[17],
            'tackles_won':x[18],
            'interceptions':x[19],
            'own_goals':x[20],
            'pens_won':x[21],
            'pens_conceded':x[22]
        })
    return players

pKeeper='<a href="/en/players/([^/]+)/">([^<]+)</a></th><td class="left poptip" data-stat="nationality" data-tip="birth" ><a[^>]+><span style="white-space: nowrap"><span[^>]+>[^<]+</span>([^<]+)</span></a></td><td class="center " data-stat="age" >([^<]+)</td><td class="right " data-stat="minutes"[^>]+>(\d*)</td><td class="right iz group_start" data-stat="shots_on_target_against" >(\d*)</td><td class="right iz" data-stat="goals_against_gk" >(\d*)</td><td class="right iz" data-stat="saves" >(\d*)</td>'

pPlayer='<a href="/en/players/([^/]+)/([^"]+)">([^<]+)</a></th><td class="right[^"]*" data-stat="shirtnumber" >[^<]+</td><td class="left poptip" data-stat="nationality"[^>]+><a[^>]+><span style="white-space: nowrap"><span[^>]+>[^<]+</span>([^<]+)</span></a></td><td class="center " data-stat="position" >([^<]+)</td><td class="center[^"]+" data-stat="age" >([^<]+)</td><td class="right[^"]*" data-stat="minutes" >([^<]+)</td><td class="right[^"]*" data-stat="goals" >([^<]+)</td><td class="right[^"]*" data-stat="assists" >([^<]+)</td><td class="right[^"]*" data-stat="pens_made" >([^<]*)</td><td class="right[^"]*" data-stat="pens_att" >([^<]*)</td><td class="right[^"]*" data-stat="shots_total" >([^<]*)</td><td class="right[^"]*" data-stat="shots_on_target" >([^<]*)</td><td class="right[^"]*" data-stat="cards_yellow" >([^<]*)</td><td class="right[^"]*" data-stat="cards_red" >([^<]*)</td><td class="right[^"]*" data-stat="fouls" >([^<]*)</td><td class="right[^"]*" data-stat="fouled" >([^<]*)</td><td class="right[^"]*" data-stat="offsides" >([^<]*)</td><td class="right[^"]*" data-stat="crosses" >([^<]*)</td><td class="right[^"]*" data-stat="tackles_won" >([^<]*)</td><td class="right[^"]*" data-stat="interceptions" >([^<]*)</td><td class="right[^"]*" data-stat="own_goals" >([^<]*)</td><td class="right[^"]*" data-stat="pens_won" >([^<]*)</td><td class="right[^"]*" data-stat="pens_conceded" >([^<]*)</td></tr>'

In [82]:
matches=[]
players=[]
goalkeepers=[]
match={}
file_name='html/0000ec37.htm'
match['mid']=file_name.split('/')[1].replace('.htm','')
with open(file_name, 'r', encoding='utf8') as f:
    html=f.read()
html=html.replace('&nbsp;&nbsp;&nbsp;','')
html=inbetween(html,'<div id="content" role="main" class="box">','<h4>About FBref.com</h4>')
scoreboards=inbetween(html,'<div class="scorebox">','<div class="scorebox_meta">')
parts=scoreboards.split('<div itemscope="" itemprop="performer"')[1:]
match['manager1']=getone(r'<strong>Manager</strong>: ([^<]+)</div>', parts[0])
match['manager2']=getone(r'<strong>Manager</strong>: ([^<]+)</div>', parts[1])
match['squad_link1']=getone(r'<a href="([^"]+)" itemprop="name">', parts[0])
match['tid1']=match['squad_link1'].split('/')[3]
match['squad_link2']=getone(r'<a href="([^"]+)" itemprop="name">', parts[1])
match['tid2']=match['squad_link2'].split('/')[3]
match['team1']=getone(r' itemprop="name">([^<]+)</a>', parts[0])
match['team2']=getone(r' itemprop="name">([^<]+)</a>', parts[1])

match['sc1']=getone(r'<div class="score">([^<]+)</div>', parts[0])
match['sc2']=getone(r'<div class="score">([^<]+)</div>', parts[1])

match['form1']=getone(r'</div><div>([^<]+)</div>', parts[0])
match['form2']=getone(r'</div><div>([^<]+)</div>', parts[1])

scorebox=inbetween(html,'<div class="scorebox_meta">','<div class="event" id="a">')
match['ds_venue']=getone(r'data-venue-epoch="(\d+)">', scorebox)
match['competition']=getone(r'<a href="/en/comps/\d+/\d+/([^"]+)">', scorebox)
match['attendance']=getone(r'<small>Attendance</small></strong>: <small>(\d+)</small>', scorebox)
match['venue']=getone(r'<small>Venue</small></strong>: <small>([^<]+)', scorebox)
match['referee']=getone(r'<span style="display:inline-block">([^\()]+) \(Referee\)</span>', scorebox)
match['ar1']=getone(r'<span style="display:inline-block">([^\()]+) \(AR1\)</span>', scorebox)
match['ar2']=getone(r'<span style="display:inline-block">([^\()]+) \(AR2\)</span>', scorebox)
match['ar3']=getone(r'<span style="display:inline-block">([^\()]+) \(4th\)</span>', scorebox)

matches.append(match)

tstats=inbetween(html,'all_stats_'+match['tid1'],'all_stats_'+match['tid2'])
tstats,keeper=tstats.split('<div id="all_keeper_stats')

mm=re.findall(pPlayer, tstats)
if len(mm)>0:
    players+=pl(mm,tid1)
m=re.search(pKeeper, keeper)
if m:
    goalkeepers.append(gk(m,tid1))
tstats=inbetween(html,'all_stats_'+match['tid2'],'</tbody></table>')
tstats,keeper=tstats.split('<div id="all_keeper_stats')
mm=re.findall(pPlayer, tstats)
if len(mm)>0:
    players+=pl(mm,tid2)
m=re.search(pKeeper, keeper)
if m:
    goalkeepers.append(gk(m,tid2))

<div id="content" role="main" class="box"> <h4>About FBref.com</h4>
<div class="scorebox"> <div class="scorebox_meta">
<div class="scorebox_meta"> <div class="event" id="a">
all_stats_4792ab67 all_stats_38154f92
all_stats_38154f92 </tbody></table>


In [83]:
pd.DataFrame(matches)

,mid,manager1,manager2,squad_link1,tid1,squad_link2,tid2,team1,team2,sc1,...,form1,form2,ds_venue,competition,attendance,venue,referee,ar1,ar2,ar3
0,0000ec37,John Askey,Billy Heath,/en/squads/4792ab67/2017-2018/Macclesfield-Tow...,4792ab67,/en/squads/38154f92/2017-2018/FC-Halifax-Town-...,38154f92,Macclesfield Town,FC Halifax Town,2,...,14-6-6,8-8-10,1514300400,2017-2018-National-League-Stats,,"Moss Rose Ground, Macclesfield",Peter Gibbons,,,


In [88]:
from fbref_parser import FbrefParser

fp=FbrefParser()
fp.parse_matches()

  0%|          | 1/89914 [00:00<53:56, 27.78it/s]00012c13.htm



# OP

In [ ]:
from op_parser import OpParser

op=OpParser()
op.parse_days()

In [ ]:
op.DATA

In [ ]:
pd.DataFrame(op.DATA)

In [ ]:
df_fbref=pd.read_csv('data/fbref/matches.csv', index_col=None)

In [ ]:
from data_provider import DataProvider
dp=DataProvider()
dp.load_op_matches()

In [ ]:
df_fbref.country.unique()

In [ ]:
df_fbref[df_fbref.champ=='Premier League 2 — Division 1']

In [ ]:
countries={'Netherlands' : 'nl','France' : 'fr','Bolivia' : 'bo','England' : 'eng','Ukraine' : 'ua','Russia' : 'ru','USA' : 'us','Spain' : 'es','Belgium' : 'be','Germany' : 'de','Sweden' : 'se','Portugal' : 'pt','Argentina' : 'ar','Finland' : 'fi','India' : 'in','Denmark' : 'dk','Uruguay' : 'uy','Venezuela' : 've','Scotland' : 'sco','Italy' : 'it','Mexico' : 'mx','Peru' : 'pe','Japan' : 'jp','Brazil' : 'br','Saudi Arabia' : 'sa','Serbia' : 'rs','Poland' : 'pl','South Korea' : 'kr','Czech Republic' : 'cz','Ecuador' : 'ec','Croatia' : 'hr','Turkey' : 'tr','Paraguay' : 'py','Romania' : 'ro','Chile' : 'cl','Colombia' : 'co','Australia' : 'au','Austria' : 'at','China' : 'cn','Greece' : 'gr','Norway' : 'no','Bulgaria' : 'bg','Switzerland' : 'ch','Ireland' : 'ir','Hungary' : 'hu','Zambia' : 'za','Canada' : 'ca'}

avoid_countries=['Albania', 'Algeria', 'Andorra', 'Angola','Armenia', 'Aruba', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus','Benin', 'Bosnia and Herzegovina','Botswana', 'Burkina Faso', 'Burundi','Cambodia', 'Cameroon','Costa Rica', 'Curaçao', 'Cyprus','DR Congo', 'Dominican Republic', 'Egypt','El Salvador', 'Estonia', 'Ethiopia','Faroe Islands', 'Fiji', 'Gabon', 'Gambia','Georgia', 'Ghana', 'Gibraltar', 'Guatemala','Haiti', 'Honduras', 'Hong Kong', 'Iceland','Indonesia', 'Iraq', 'Israel','Ivory Coast', 'Jamaica', 'Jordan', 'Kazakhstan', 'Kenya','Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon','Lesotho', 'Liberia', 'Liechtenstein', 'Lithuania', 'Luxembourg','Macao', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta','Mauritania', 'Moldova', 'Mongolia', 'Montenegro','Morocco', 'Mozambique', 'Myanmar', 'Namibia','New Zealand', 'Nicaragua', 'Nigeria','North Macedonia', 'Northern Ireland', 'Oman','Pakistan', 'Palestine', 'Panama','Philippines', 'Qatar','Republic of the Congo', 'Rwanda','San Marino', 'Senegal','Serbia and Montenegro', 'Singapore', 'Slovakia', 'Slovenia','South Africa', 'Sri Lanka', 'Sudan', 'Syria', 'Taiwan','Tajikistan', 'Tanzania', 'Thailand', 'Togo','Trinidad and Tobago', 'Tunisia', 'Turkmenistan','Uganda', 'United Arab Emirates','Uzbekistan', 'Vietnam', 'Wales', 'Yemen','Zimbabwe']

In [ ]:
champs=[]
pCountry=r'</span>([^<]+)</a></th>'
pChamp=r'<a foo="f" href="([^"]+)">([^<]+)</a>'
with open('html/op.html', 'r', encoding='utf8') as f:
    html=f.read()
parts=html.split('<th class="first2 tl" colspan="2">')[1:]
for part in parts:
    m=re.search(pCountry, part)
    if m:
        country=m[1]
    mm=re.findall(pChamp, part)
    if len(mm)>0:
        for x in mm:
            champs.append({
                'country':country,
                'champ':x[1],
                'link':x[0]
            })
df_champs=pd.DataFrame(champs)

In [ ]:
df_champs=df_champs.loc[~df_champs.country.isin(avoid_countries)]

In [ ]:
df_champs.to_csv('data/op/champs1.csv', index=False)

In [ ]:
df_champs=pd.read_csv('data/op/champs.csv', index_col=None)

In [ ]:
df_champs[df_champs['active']==1].to_csv('data/op/champs1.csv', index=False)

In [ ]:
df_champs['active']=df_champs['country'].apply(lambda x: 1 if '#' in x else 0)

In [ ]:
df_champs['country']=df_champs['country'].apply(lambda x: x.replace('#',''))

In [ ]:
with open('html/bookies.json', 'r', encoding='utf8') as f:
    txt=f.read()
jb=json.loads(txt)

In [18]:
bookies=[{
    'bid':int(jb[b]['idProvider']),
    'name':jb[b]['WebName'],
    'url':jb[b]['WebUrl'],
    'link':jb[b]['Url'],
    'isbookie':jb[b]['IsBookmaker']=='y'
    } for b in jb]

In [19]:
df_bookies=pd.DataFrame(bookies)
df_bookies.to_csv('data/op/bookies.csv', index=False)

## Match

In [119]:
file_name='html/EFEkIb54.json'
with open(file_name, 'r', encoding='utf8') as f:
    html=f.read()
js=json.loads(html)
mid=file_name.split('/')[1].replace('.json','')
# 0 - w1
# 1 - x
# 2 - w2


In [120]:
odds=js['d']['oddsdata']['back']['E-1-2-0-0-0']['odds']
movement=js['d']['oddsdata']['back']['E-1-2-0-0-0']['movement']
opening_odds=js['d']['oddsdata']['back']['E-1-2-0-0-0']['opening_odds']
opening_change_time=js['d']['oddsdata']['back']['E-1-2-0-0-0']['opening_change_time']
opening_volume=js['d']['oddsdata']['back']['E-1-2-0-0-0']['opening_volume']
volume=js['d']['oddsdata']['back']['E-1-2-0-0-0']['volume']
change_time=js['d']['oddsdata']['back']['E-1-2-0-0-0']['change_time']

In [121]:
bookies={}
for x in odds:
    bookies[x]={
        'mid':mid,
        'bid':x,
        'w1':odds[x]['0'],
        'wx':odds[x]['1'],
        'w2':odds[x]['2']
    }

for x in movement:
    bookies[x]['move_1']=str(movement[x]['0'])[0].upper()
    bookies[x]['move_x']=str(movement[x]['1'])[0].upper()
    bookies[x]['move_2']=str(movement[x]['2'])[0].upper()

for x in opening_odds:
    bookies[x]['open_1']=opening_odds[x]['0']
    bookies[x]['open_x']=opening_odds[x]['1']
    bookies[x]['open_2']=opening_odds[x]['2']

for x in opening_change_time:
    bookies[x]['time_open']=max([opening_change_time[x]['0'],opening_change_time[x]['1'],opening_change_time[x]['2']])

for x in change_time:
    bookies[x]['time_close']=max([change_time[x]['0'],change_time[x]['1'],change_time[x]['2']])


In [122]:
pd.DataFrame([bookies[x] for x in bookies])


,mid,bid,w1,wx,w2,move_1,move_x,move_2,open_1,open_x,open_2,time_open,time_close
0,EFEkIb54,417,2.36,3.14,3.34,D,D,U,3.18,3.20,2.36,1552887983,1553629298
1,EFEkIb54,453,2.36,3.14,3.34,D,D,U,3.18,3.20,2.36,1552888069,1553629352
2,EFEkIb54,9,2.44,3.20,3.37,D,D,U,3.27,3.30,2.43,1552983711,1553629497
3,EFEkIb54,32,2.49,3.17,3.30,D,D,U,3.27,3.30,2.43,1552983720,1553629038
4,EFEkIb54,141,2.52,3.05,3.00,D,U,U,3.00,3.00,2.25,1552983754,1553596748
5,EFEkIb54,160,2.52,3.05,2.95,D,D,U,3.10,3.15,2.33,1552984330,1553585881
6,EFEkIb54,73,2.46,3.20,3.32,D,D,U,3.29,3.34,2.40,1552984587,1553629318
7,EFEkIb54,455,2.46,3.20,3.32,D,D,U,3.29,3.34,2.40,1552984594,1553629322
8,EFEkIb54,454,2.49,3.07,3.10,D,D,U,3.17,3.20,2.37,1552984740,1553629065
9,EFEkIb54,429,2.55,3.10,3.00,D,U,U,3.00,3.00,2.20,1552985819,1553622736


In [127]:
from op_parser import OpParser

op=OpParser()
op.parse_matches()

In [128]:
op.DATA

[         mid  bid    w1    wx    w2 move_1 move_x move_2  open_1  open_x  \
 0   003ZibWr    1  1.90  3.20  3.40      N      N      N     NaN     NaN   
 1   003ZibWr    2  1.95  3.30  3.50      D      U      U    2.00    3.25   
 2   003ZibWr    3  1.80  3.40  3.55      D      U      U    1.95    3.20   
 3   003ZibWr    5  1.85  3.40  3.60      N      N      N    1.85    3.40   
 4   003ZibWr    9  1.85  3.25  3.70      N      N      N     NaN     NaN   
 5   003ZibWr   14  1.85  3.32  3.71      D      U      U    1.94    3.24   
 6   003ZibWr   15  1.80  3.50  4.00      N      N      N     NaN     NaN   
 7   003ZibWr   16  1.91  3.60  3.80      N      U      D    1.91    3.40   
 8   003ZibWr   24  1.80  3.50  3.70      D      U      U    1.88    3.35   
 9   003ZibWr   26  1.95  3.20  3.60      U      D      D    1.85    3.30   
 10  003ZibWr   30  1.91  3.25  3.40      U      N      D    1.83    3.25   
 11  003ZibWr   32  1.75  3.50  3.75      D      U      U    1.95    3.20   